This notebook performs regenie preprocessing, step 1 and 2 using Swiss Army Knife (SAK) for the dementia & age-stratified GWASes. 
- Note: to rerun it with different phenotype/genotype data, don't forget to download, edit and re-upload the scripts in SAK_scripts folder 

### Setup

In [6]:
# %%bash
# pip install openpyxl
# pip install -U scikit-learn
# pip install statsmodels

In [3]:
import os
import glob
import pandas as pd
from pandas.core.common import flatten
import re
import numpy as np
import seaborn as sns
from sklearn import datasets, linear_model, metrics
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Make merged genotypes files 

- To be used for all GWAS analyses
- Write the <code>run_merge.sh</code> script first, and pass it to Army Knife (SAK)

In [41]:
%%bash
dx upload run_merge.sh --dest vasilis/SAK_scripts/

ID                                file-GxQjFq0JZB73Zxz4pxzf4f9f
Class                             file
Project                           project-GfvP6PQJZB72v2Vk348Bb2yg
Folder                            /vasilis/SAK_scripts
Name                              run_merge.sh
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Tue Dec 17 14:08:44 2024
Created by                        vasilisraptis
 via the job                      job-GxQbFp0JZB795yVgB4fxJq25
Last modified                     Tue Dec 17 14:08:45 2024
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-dnanexus"


In [42]:
%%bash
dx run swiss-army-knife \
    -iin="vasilis/SAK_scripts/run_merge.sh" \
    -icmd="sh run_merge.sh" \
    --tag="merge" \
    --instance-type "mem1_ssd1_v2_x16" \
    --destination="vasilis/data/GWAS/genotypes/" \
    --brief --yes --priority high

job-GxQjG0QJZB740K5G53vvZGvG


### Dementia stratified

#### Genotype QC

- Use plink2 with Swiss Army Knife (SAK) to perform pre-step1 genotype filtering. <br>
- See here: https://dnanexus.gitbook.io/uk-biobank-rap/getting-started/training-videos/analysis-and-data-types#gwas-on-ukb-rap-using-regenie
- Write the <code>1_regenie_pre_step1_qc.sh</code> script first, and pass it to SAK

In [77]:
## make id keep file: FID IID
#!awk 'BEGIN { FS="," } NR>1 {print $1,$2}' demgroup_ukb_wb_del_cov_e4_dem_phe.csv > demgroup_ukb_wb.ids # dementia group
#!dx upload demgroup_ukb_wb.ids --dest vasilis/data/GWAS/sensitivity/dementia_stratified/

!awk 'BEGIN { FS="," } NR>1 {print $1,$2}' nodemgroup_ukb_wb_del_cov_e4_dem_phe.csv > nodemgroup_ukb_wb.ids # non-dementia group
!dx upload nodemgroup_ukb_wb.ids --dest vasilis/data/GWAS/sensitivity/dementia_stratified/


[===========================================================>] Uploaded 6,392,928 of 6,392,928 bytes (100%) nodemgroup_ukb_wb.ids
ID                                file-GxQjp6QJZB7057qvf7QgZ9B8
Class                             file
Project                           project-GfvP6PQJZB72v2Vk348Bb2yg
Folder                            /vasilis/data/GWAS/sensitivity/dementia_stratified
Name                              nodemgroup_ukb_wb.ids
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Tue Dec 17 14:39:22 2024
Created by                        vasilisraptis
 via the job                      job-GxQbFp0JZB795yVgB4fxJq25
Last modified                     Tue Dec 17 14:39:23 2024
Media type                        
archivalState                     "live"
cloudAccount              

In [80]:
%%bash
dx upload 1_regenie_pre_step1_qc.sh --dest vasilis/SAK_scripts/

ID                                file-GxQk810JZB7PVx763P83pY0F
Class                             file
Project                           project-GfvP6PQJZB72v2Vk348Bb2yg
Folder                            /vasilis/SAK_scripts
Name                              1_regenie_pre_step1_qc.sh
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Tue Dec 17 15:08:52 2024
Created by                        vasilisraptis
 via the job                      job-GxQbFp0JZB795yVgB4fxJq25
Last modified                     Tue Dec 17 15:08:53 2024
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-dnanexus"


In [95]:
%%bash
keep_dem="demgroup"
keep_nodem="nodemgroup"
gendir="vasilis/data/GWAS/genotypes"
dest="vasilis/data/GWAS/sensitivity/dementia_stratified"

# dementia
dx run swiss-army-knife \
    -iin="vasilis/SAK_scripts/1_regenie_pre_step1_qc.sh" \
    -iin="${gendir}/ukb22418_allChrs.bed" \
    -iin="${gendir}/ukb22418_allChrs.bim" \
    -iin="${gendir}/ukb22418_allChrs.fam" \
    -iin="${dest}/demgroup_ukb_wb.ids" \
    -iin="${dest}/nodemgroup_ukb_wb.ids" \
    -icmd="sh 1_regenie_pre_step1_qc.sh ${keep_dem}" \
    --tag="dem_qc" \
    --instance-type "mem1_ssd1_v2_x16" \
    --destination="${dest}/" \
    --brief --yes --priority high

# non-dementia
dx run swiss-army-knife \
    -iin="vasilis/SAK_scripts/1_regenie_pre_step1_qc.sh" \
    -iin="${gendir}/ukb22418_allChrs.bed" \
    -iin="${gendir}/ukb22418_allChrs.bim" \
    -iin="${gendir}/ukb22418_allChrs.fam" \
    -iin="${dest}/demgroup_ukb_wb.ids" \
    -iin="${dest}/nodemgroup_ukb_wb.ids" \
    -icmd="sh 1_regenie_pre_step1_qc.sh ${keep_nodem}" \
    --tag="nodem_qc" \
    --instance-type "mem1_ssd1_v2_x16" \
    --destination="${dest}/" \
    --brief --yes --priority high


job-GxQp6kjJZB71XQf6bYZ1Q259
job-GxQp6p0JZB7793GxkQ0b6432


#### Step 1

In [31]:
%%bash
dx upload 2_regenie_step1.sh --dest vasilis/SAK_scripts/

ID                                file-GxVFxYjJZB7K7kX40Vf6pFKX
Class                             file
Project                           project-GfvP6PQJZB72v2Vk348Bb2yg
Folder                            /vasilis/SAK_scripts
Name                              2_regenie_step1.sh
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Wed Dec 18 12:23:43 2024
Created by                        vasilisraptis
 via the job                      job-GxVF8PjJZB72fPJFGZ6FK56F
Last modified                     Wed Dec 18 12:23:44 2024
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-dnanexus"


In [115]:
## make space seperated pheno file
# tempdem = pd.read_csv("demgroup_ukb_wb_del_cov_e4_dem_phe.csv")
# tempdem.to_csv("demgroup_ukb_wb_del_cov_e4_dem_phe.txt", sep=" ", index=False)

# tempnodem = pd.read_csv("nodemgroup_ukb_wb_del_cov_e4_dem_phe.csv")
# tempnodem.to_csv("nodemgroup_ukb_wb_del_cov_e4_dem_phe.txt", sep=" ", index=False)

# !dx upload demgroup_ukb_wb_del_cov_e4_dem_phe.txt --dest vasilis/data/GWAS/sensitivity/dementia_stratified/
# !dx upload nodemgroup_ukb_wb_del_cov_e4_dem_phe.txt --dest vasilis/data/GWAS/sensitivity/dementia_stratified/


In [32]:
%%bash
keep_dem="demgroup"
keep_nodem="nodemgroup"
gendir="vasilis/data/GWAS/genotypes"
datadir="vasilis/data/GWAS/sensitivity/dementia_stratified"
dest="vasilis/data/GWAS/sensitivity/dementia_stratified/step1/"

# dementia
dx run swiss-army-knife \
    -iin="vasilis/SAK_scripts/2_regenie_step1.sh" \
    -iin="${gendir}/ukb22418_allChrs.bed" \
    -iin="${gendir}/ukb22418_allChrs.bim" \
    -iin="${gendir}/ukb22418_allChrs.fam" \
    -iin="${datadir}/demgroup_ukb_wb_del_cov_e4_dem_phe.txt" \
    -iin="${datadir}/qc_pass_called_${keep_dem}.id" \
    -iin="${datadir}/qc_pass_called_${keep_dem}.snplist" \
    -iin="${datadir}/qc_pass_called_${keep_nodem}.id" \
    -iin="${datadir}/qc_pass_called_${keep_nodem}.snplist" \
    -icmd="sh 2_regenie_step1.sh ${keep_dem}" \
    --tag="dem_step1" \
    --instance-type "mem1_ssd1_v2_x16" \
    --destination="${dest}" \
    --brief --yes --priority high

# non-dementia
dx run swiss-army-knife \
    -iin="vasilis/SAK_scripts/2_regenie_step1.sh" \
    -iin="${gendir}/ukb22418_allChrs.bed" \
    -iin="${gendir}/ukb22418_allChrs.bim" \
    -iin="${gendir}/ukb22418_allChrs.fam" \
    -iin="${datadir}/nodemgroup_ukb_wb_del_cov_e4_dem_phe.txt" \
    -iin="${datadir}/qc_pass_called_${keep_dem}.id" \
    -iin="${datadir}/qc_pass_called_${keep_dem}.snplist" \
    -iin="${datadir}/qc_pass_called_${keep_nodem}.id" \
    -iin="${datadir}/qc_pass_called_${keep_nodem}.snplist" \
    -icmd="sh 2_regenie_step1.sh ${keep_nodem}" \
    --tag="nodem_step1" \
    --instance-type "mem1_ssd1_v2_x16" \
    --destination="${dest}" \
    --brief --yes --priority high


job-GxVFxxQJZB7FpZjvygQK2p1g
job-GxVFxxjJZB7K7kX40Vf6pFZ2


#### Step 2

In [39]:
%%bash
dx upload 3_regenie_step2.sh --dest vasilis/SAK_scripts/

ID                                file-GxVJJFjJZB70xpB3FxP3x9fy
Class                             file
Project                           project-GfvP6PQJZB72v2Vk348Bb2yg
Folder                            /vasilis/SAK_scripts
Name                              3_regenie_step2.sh
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Wed Dec 18 14:05:35 2024
Created by                        vasilisraptis
 via the job                      job-GxVF8PjJZB72fPJFGZ6FK56F
Last modified                     Wed Dec 18 14:05:36 2024
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-dnanexus"


In [46]:
# %%bash 
# dx download -f vasilis/data/GWAS/sensitivity/dementia_stratified/step1/ukb_wb_demgroup_pred.list
# cat ukb_wb_demgroup_pred.list | sed 's/out\/out/in\/in/g' > ukb_wb_demgroup_fixed_pred.list
# dx upload ukb_wb_demgroup_fixed_pred.list --dest vasilis/data/GWAS/sensitivity/dementia_stratified/step1/

# dx download -f vasilis/data/GWAS/sensitivity/dementia_stratified/step1/ukb_wb_nodemgroup_pred.list
# cat ukb_wb_nodemgroup_pred.list | sed 's/out\/out/in\/in/g' > ukb_wb_nodemgroup_fixed_pred.list
# dx upload ukb_wb_nodemgroup_fixed_pred.list --dest vasilis/data/GWAS/sensitivity/dementia_stratified/step1/


ID                                file-GxVJQGQJZB74fvB767Zz42Gk
Class                             file
Project                           project-GfvP6PQJZB72v2Vk348Bb2yg
Folder                            /vasilis/data/GWAS/sensitivity/dementia_stratified/step1
Name                              ukb_wb_demgroup_fixed_pred.list
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Wed Dec 18 14:12:02 2024
Created by                        vasilisraptis
 via the job                      job-GxVF8PjJZB72fPJFGZ6FK56F
Last modified                     Wed Dec 18 14:12:03 2024
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-dnanexus"


In [ ]:
%%bash
keep_dem="demgroup"
keep_nodem="nodemgroup"
bgen_dir="/Bulk/Imputation/UKB imputation from genotype"
datadir="vasilis/data/GWAS/sensitivity/dementia_stratified"
dest="vasilis/data/GWAS/sensitivity/dementia_stratified/step2/"

for CHR in {1..22} X; do
    # dementia
    # dx run swiss-army-knife \
    #     -iin="vasilis/SAK_scripts/3_regenie_step2.sh" \
    #     -iin="${bgen_dir}/ukb22828_c${CHR}_b0_v3.bgen" \
    #     -iin="${bgen_dir}/ukb22828_c${CHR}_b0_v3.sample" \
    #     -iin="${datadir}/demgroup_ukb_wb_del_cov_e4_dem_phe.txt" \
    #     -iin="${datadir}/qc_pass_called_${keep_dem}.id" \
    #     -iin="${datadir}/qc_pass_called_${keep_dem}.snplist" \
    #     -iin="${datadir}/qc_pass_called_${keep_nodem}.id" \
    #     -iin="${datadir}/qc_pass_called_${keep_nodem}.snplist" \
    #     -iin="${datadir}/step1/ukb_wb_${keep_dem}_1.loco" \
    #     -iin="${datadir}/step1/ukb_wb_${keep_dem}_pred.list" \
    #     -icmd="sh 3_regenie_step2.sh ${keep_dem} ${CHR}" \
    #     --tag="dem_step2" \
    #     --instance-type "mem1_ssd1_v2_x16" \
    #     --destination="${dest}" \
    #     --brief --yes --priority high

    # non-dementia
    dx run swiss-army-knife \
        -iin="vasilis/SAK_scripts/3_regenie_step2.sh" \
        -iin="${bgen_dir}/ukb22828_c${CHR}_b0_v3.bgen" \
        -iin="${bgen_dir}/ukb22828_c${CHR}_b0_v3.sample" \
        -iin="${datadir}/nodemgroup_ukb_wb_del_cov_e4_dem_phe.txt" \
        -iin="${datadir}/qc_pass_called_${keep_dem}.id" \
        -iin="${datadir}/qc_pass_called_${keep_dem}.snplist" \
        -iin="${datadir}/qc_pass_called_${keep_nodem}.id" \
        -iin="${datadir}/qc_pass_called_${keep_nodem}.snplist" \
        -iin="${datadir}/step1/ukb_wb_${keep_nodem}_1.loco" \
        -iin="${datadir}/step1/ukb_wb_${keep_nodem}_pred.list" \
        -icmd="sh 3_regenie_step2.sh ${keep_nodem} ${CHR}" \
        --tag="nd_step2_${CHR}" \
        --instance-type "mem1_ssd1_v2_x16" \
        --destination="${dest}" \
        --brief --yes --priority high
done


job-GxVz7zQJZB7Fjj88b9zz6pyY
job-GxVz7zjJZB7Fjj88b9zz6pyq
job-GxVz800JZB7KVg02bZKK12jb
job-GxVz808JZB7Fjj88b9zz6pyx
job-GxVz80QJZB7F39Kkbfg94k71
job-GxVz80jJZB7Fjj88b9zz6pz3
job-GxVz810JZB74B1ybZ758k72x
job-GxVz818JZB7KVg02bZKK12kq
job-GxVz81QJZB7F39Kkbfg94k7F
job-GxVz81jJZB7KxGbQXQpG2Z06
job-GxVz828JZB7F39Kkbfg94k7P
job-GxVz82QJZB74B1ybZ758k73F
job-GxVz82jJZB7F39Kkbfg94k7v
job-GxVz830JZB7KVg02bZKK12pX
job-GxVz838JZB7KVg02bZKK12pj
job-GxVz83QJZB7F39Kkbfg94k8Z
job-GxVz83jJZB7Fjj88b9zz6q73
job-GxVz840JZB7KxGbQXQpG2Z3V
job-GxVz848JZB7F39Kkbfg94k91
job-GxVz84QJZB7F39Kkbfg94k99
job-GxVz84jJZB7F39Kkbfg94k9F
job-GxVz850JZB7KyzbjZJxxbYYf


### Age stratified

#### Genotype QC

- Use plink2 with Swiss Army Knife (SAK) to perform pre-step1 genotype filtering. <br>
- See here: https://dnanexus.gitbook.io/uk-biobank-rap/getting-started/training-videos/analysis-and-data-types#gwas-on-ukb-rap-using-regenie
- Write the <code>1_regenie_pre_step1_qc.sh</code> script first, and pass it to SAK

In [94]:
## make id keep file: FID IID
!awk 'BEGIN { FS="," } NR>1 {print $1,$2}' age60plus_ukb_wb_del_cov_e4_dem_phe.csv > age60plus_ukb_wb.ids # 60+ group
!dx upload age60plus_ukb_wb.ids --dest vasilis/data/GWAS/sensitivity/age_stratified/

!awk 'BEGIN { FS="," } NR>1 {print $1,$2}' age60minus_ukb_wb_del_cov_e4_dem_phe.csv > age60minus_ukb_wb.ids # 60- group
!dx upload age60minus_ukb_wb.ids --dest vasilis/data/GWAS/sensitivity/age_stratified/


In [99]:
%%bash
keep_60plus="age60plus"
keep_60minus="age60minus"
gendir="vasilis/data/GWAS/genotypes"
dest="vasilis/data/GWAS/sensitivity/age_stratified"

# 60+
dx run swiss-army-knife \
    -iin="vasilis/SAK_scripts/1_regenie_pre_step1_qc.sh" \
    -iin="${gendir}/ukb22418_allChrs.bed" \
    -iin="${gendir}/ukb22418_allChrs.bim" \
    -iin="${gendir}/ukb22418_allChrs.fam" \
    -iin="${dest}/age60plus_ukb_wb.ids" \
    -iin="${dest}/age60minus_ukb_wb.ids" \
    -icmd="sh 1_regenie_pre_step1_qc.sh ${keep_60plus}" \
    --tag="old_qc" \
    --instance-type "mem1_ssd1_v2_x16" \
    --destination="${dest}/" \
    --brief --yes --priority high

# 60-
dx run swiss-army-knife \
    -iin="vasilis/SAK_scripts/1_regenie_pre_step1_qc.sh" \
    -iin="${gendir}/ukb22418_allChrs.bed" \
    -iin="${gendir}/ukb22418_allChrs.bim" \
    -iin="${gendir}/ukb22418_allChrs.fam" \
    -iin="${dest}/age60plus_ukb_wb.ids" \
    -iin="${dest}/age60minus_ukb_wb.ids" \
    -icmd="sh 1_regenie_pre_step1_qc.sh ${keep_60minus}" \
    --tag="young_qc" \
    --instance-type "mem1_ssd1_v2_x16" \
    --destination="${dest}/" \
    --brief --yes --priority high


job-GxQpjB8JZB75Pj4f5b1YqJg9
job-GxQpjBQJZB75Pj4f5b1YqJgF


#### Step 1

In [114]:
## make space seperated pheno file
# tempold= pd.read_csv("age60plus_ukb_wb_del_cov_e4_dem_phe.csv")
# tempold.to_csv("age60plus_ukb_wb_del_cov_e4_dem_phe.txt", sep=" ", index=False)

# tempyoung = pd.read_csv("age60minus_ukb_wb_del_cov_e4_dem_phe.csv")
# tempyoung.to_csv("age60minus_ukb_wb_del_cov_e4_dem_phe.txt", sep=" ", index=False)

# !dx upload age60plus_ukb_wb_del_cov_e4_dem_phe.txt --dest vasilis/data/GWAS/sensitivity/age_stratified/
# !dx upload age60minus_ukb_wb_del_cov_e4_dem_phe.txt --dest vasilis/data/GWAS/sensitivity/age_stratified/


In [33]:
%%bash
keep_60plus="age60plus"
keep_60minus="age60minus"
gendir="vasilis/data/GWAS/genotypes"
datadir="vasilis/data/GWAS/sensitivity/age_stratified"
dest="vasilis/data/GWAS/sensitivity/age_stratified/step1/"

# 60+
dx run swiss-army-knife \
    -iin="vasilis/SAK_scripts/2_regenie_step1.sh" \
    -iin="${gendir}/ukb22418_allChrs.bed" \
    -iin="${gendir}/ukb22418_allChrs.bim" \
    -iin="${gendir}/ukb22418_allChrs.fam" \
    -iin="${datadir}/${keep_60plus}_ukb_wb_del_cov_e4_dem_phe.txt" \
    -iin="${datadir}/${keep_60minus}_ukb_wb_del_cov_e4_dem_phe.txt" \
    -iin="${datadir}/qc_pass_called_${keep_60plus}.id" \
    -iin="${datadir}/qc_pass_called_${keep_60plus}.snplist" \
    -iin="${datadir}/qc_pass_called_${keep_60minus}.id" \
    -iin="${datadir}/qc_pass_called_${keep_60minus}.snplist" \
    -icmd="sh 2_regenie_step1.sh ${keep_60plus}" \
    --tag="old_step1" \
    --instance-type "mem1_ssd1_v2_x16" \
    --destination="${dest}" \
    --brief --yes --priority high

# 60-
dx run swiss-army-knife \
    -iin="vasilis/SAK_scripts/2_regenie_step1.sh" \
    -iin="${gendir}/ukb22418_allChrs.bed" \
    -iin="${gendir}/ukb22418_allChrs.bim" \
    -iin="${gendir}/ukb22418_allChrs.fam" \
    -iin="${datadir}/${keep_60plus}_ukb_wb_del_cov_e4_dem_phe.txt" \
    -iin="${datadir}/${keep_60minus}_ukb_wb_del_cov_e4_dem_phe.txt" \
    -iin="${datadir}/qc_pass_called_${keep_60plus}.id" \
    -iin="${datadir}/qc_pass_called_${keep_60plus}.snplist" \
    -iin="${datadir}/qc_pass_called_${keep_60minus}.id" \
    -iin="${datadir}/qc_pass_called_${keep_60minus}.snplist" \
    -icmd="sh 2_regenie_step1.sh ${keep_60minus}" \
    --tag="young_step1" \
    --instance-type "mem1_ssd1_v2_x16" \
    --destination="${dest}" \
    --brief --yes --priority high


job-GxVFxzQJZB7J5ZqPpY99kJz5
job-GxVFxzjJZB78FBpXjx0Qq2YJ


#### Step 2

In [49]:
%%bash
keep_60plus="age60plus"
keep_60minus="age60minus"
bgen_dir="/Bulk/Imputation/UKB imputation from genotype"
datadir="vasilis/data/GWAS/sensitivity/age_stratified"
dest="vasilis/data/GWAS/sensitivity/age_stratified/step2/"

for CHR in {1..22} X; do
    # 60+
    dx run swiss-army-knife \
        -iin="vasilis/SAK_scripts/3_regenie_step2.sh" \
        -iin="${bgen_dir}/ukb22828_c${CHR}_b0_v3.bgen" \
        -iin="${bgen_dir}/ukb22828_c${CHR}_b0_v3.sample" \
        -iin="${datadir}/${keep_60plus}_ukb_wb_del_cov_e4_dem_phe.txt" \
        -iin="${datadir}/qc_pass_called_${keep_60plus}.id" \
        -iin="${datadir}/qc_pass_called_${keep_60plus}.snplist" \
        -iin="${datadir}/qc_pass_called_${keep_60minus}.id" \
        -iin="${datadir}/qc_pass_called_${keep_60minus}.snplist" \
        -iin="${datadir}/step1/ukb_wb_${keep_60plus}_1.loco" \
        -iin="${datadir}/step1/ukb_wb_${keep_60plus}_pred.list" \
        -icmd="sh 3_regenie_step2.sh ${keep_60plus} ${CHR}" \
        --tag="old_step2_c${CHR}" \
        --instance-type "mem1_ssd1_v2_x16" \
        --destination="${dest}" \
        --brief --yes --priority high
    # # 60-
    # dx run swiss-army-knife \
    #     -iin="vasilis/SAK_scripts/3_regenie_step2.sh" \
    #     -iin="${bgen_dir}/ukb22828_c${CHR}_b0_v3.bgen" \
    #     -iin="${bgen_dir}/ukb22828_c${CHR}_b0_v3.sample" \
    #     -iin="${datadir}/${keep_60minus}_ukb_wb_del_cov_e4_dem_phe.txt" \
    #     -iin="${datadir}/qc_pass_called_${keep_60plus}.id" \
    #     -iin="${datadir}/qc_pass_called_${keep_60plus}.snplist" \
    #     -iin="${datadir}/qc_pass_called_${keep_60minus}.id" \
    #     -iin="${datadir}/qc_pass_called_${keep_60minus}.snplist" \
    #     -iin="${datadir}/step1/ukb_wb_${keep_60minus}_1.loco" \
    #     -iin="${datadir}/step1/ukb_wb_${keep_60minus}_pred.list" \
    #     -icmd="sh 3_regenie_step2.sh ${keep_60minus} ${CHR}" \
    #     --tag="y_step2_c${CHR}" \
    #     --instance-type "mem1_ssd1_v2_x16" \
    #     --destination="${dest}" \
    #     --brief --yes --priority high
done


job-GxVQ8xjJZB7985b92Q04bk99
job-GxVQ8y0JZB7K2F07q2G5j5QF
job-GxVQ8y8JZB7BpgF4q5K7f04k
job-GxVQ8yQJZB74v2GxKZy2kBYZ
job-GxVQ8yjJZB7GQxXZyzg4x10J
job-GxVQ8z0JZB76XgjK8Fzq5p1f
job-GxVQ8z8JZB7K2F07q2G5j5QY
job-GxVQ8zQJZB78VFJb195yx74B
job-GxVQ900JZB78VFJb195yx74G
job-GxVQ908JZB7G3f01PyFY80QJ
job-GxVQ90QJZB74v2GxKZy2kBYp
job-GxVQ90jJZB7BpgF4q5K7f070
job-GxVQ910JZB7K2F07q2G5j5Qk
job-GxVQ918JZB7985b92Q04bk9q
job-GxVQ91QJZB7K2F07q2G5j5Qq
job-GxVQ91jJZB7G3f01PyFY80QP
job-GxVQ920JZB7GQxXZyzg4x110
job-GxVQ928JZB7GQxXZyzg4x112
job-GxVQ92QJZB7G3f01PyFY80QV
job-GxVQ92jJZB7GQxXZyzg4x114
job-GxVQ930JZB78VFJb195yx75F
job-GxVQ938JZB78VFJb195yx75J
job-GxVQ93jJZB76XgjK8Fzq5p27
